In [4]:
import pandas as pd
import openpyxl
import json
import requests
import matplotlib.pyplot as plt

#### Read ticker symbols of stocks from etoro sheet (exported to another document)

In [6]:
file = r'D:\\lianz\Desktop\\Python\personal_projects\\personal_finance\\tickers.json'
with open(file, 'r+') as f:
    tickers = json.load(f)

In [7]:
tickers[0]

'A'

#### Read in json files (requests not made repetitively)

In [13]:
def read_json(ticker_list, statement):

    with open(f'D:\lianz\Desktop\Python\personal_projects\personal_finance\{statement}/{ticker_list}.json', 'r+') as json_file:
        x = json.load(json_file)

    return x

In [11]:
company_statements = ['income-statement',
                      'cash-flow-statement', 'balance-sheet-statement']

In [14]:
d = read_json(tickers[0],company_statements[0])

In [10]:
[d[i]['calendarYear'] for i in range(len(d))]

['2022', '2021', '2020', '2019', '2018']

In [15]:
d[0].items()

dict_items([('date', '2022-10-31'), ('symbol', 'A'), ('reportedCurrency', 'USD'), ('cik', '0001090872'), ('fillingDate', '2022-12-21'), ('acceptedDate', '2022-12-20 18:42:30'), ('calendarYear', '2022'), ('period', 'FY'), ('revenue', 6848000000), ('costOfRevenue', 3126000000), ('grossProfit', 3722000000), ('grossProfitRatio', 0.5435163551401869), ('researchAndDevelopmentExpenses', 467000000), ('generalAndAdministrativeExpenses', 0.0), ('sellingAndMarketingExpenses', 0.0), ('sellingGeneralAndAdministrativeExpenses', 1637000000), ('otherExpenses', 0.0), ('operatingExpenses', 2104000000), ('costAndExpenses', 5230000000), ('interestIncome', 9000000.0), ('interestExpense', 84000000), ('depreciationAndAmortization', 317000000), ('ebitda', 1905000000), ('ebitdaratio', 0.27818341121495327), ('operatingIncome', 1618000000), ('operatingIncomeRatio', 0.2362733644859813), ('totalOtherIncomeExpensesNet', -114000000), ('incomeBeforeTax', 1504000000), ('incomeBeforeTaxRatio', 0.21962616822429906), ('i

In [16]:
l = []
dict_holder = {}

for n in ['epsdiluted','netIncome','revenue']:
    # print(n)

    for i,x in enumerate(d[::-1]):
        dict_holder[x['calendarYear']] = x[f'{n}']
    l.append(dict_holder)
    dict_holder = {}
l

[{'2018': 0.97, '2019': 3.37, '2020': 2.3, '2021': 3.94, '2022': 4.08},
 {'2018': 316000000,
  '2019': 1071000000,
  '2020': 719000000,
  '2021': 1210000000,
  '2022': 1254000000},
 {'2018': 4914000000,
  '2019': 5163000000,
  '2020': 5339000000,
  '2021': 6319000000,
  '2022': 6848000000}]

In [17]:
eps_df = pd.DataFrame.from_records(l, index=['epsdiluted','netIncome','revenue']).T
eps_df

,epsdiluted,netIncome,revenue
2018,0.97,3.160000e+08,4.914000e+09
2019,3.37,1.071000e+09,5.163000e+09
2020,2.30,7.190000e+08,5.339000e+09
2021,3.94,1.210000e+09,6.319000e+09
2022,4.08,1.254000e+09,6.848000e+09


In [18]:
terms_interested = {'revenue': 'revenue',
                    'gross margin%': 'grossProfitRatio',
                    'Operating Income': 'operatingIncome',
                    'Operating Margin %': 'operatingIncomeRatio',
                    'Net Income': 'netIncome',
                    'Net Income Margin': 'netIncomeRatio',
                    'Earnings per Share': 'epsdiluted',
                    'Shares Oustanding (diluted)': 'weightedAverageShsOutDil'}

In [16]:
items_interested = terms_interested.values()

for items in items_interested:
    print(items)

revenue
grossProfitRatio
operatingIncome
operatingIncomeRatio
netIncome
netIncomeRatio
epsdiluted
weightedAverageShsOutDil


In [17]:
[items for items in items_interested if items in d[0].keys()]

['revenue',
 'grossProfitRatio',
 'operatingIncome',
 'operatingIncomeRatio',
 'netIncome',
 'netIncomeRatio',
 'epsdiluted',
 'weightedAverageShsOutDil']

In [18]:
arrangement = (2,3)
column_names = []

for i,nrow in enumerate(arrangement):
    for ncolumn in range(1,nrow+1):
        column_names.append(f'c_{i}_{ncolumn}')

column_names

['c_0_1', 'c_0_2', 'c_1_1', 'c_1_2', 'c_1_3']

In [19]:
eps_df.index

Index(['2018', '2019', '2020', '2021', '2022'], dtype='object')

In [20]:
eps_df


,epsdiluted,netIncome,revenue
2018,0.97,3.160000e+08,4.914000e+09
2019,3.37,1.071000e+09,5.163000e+09
2020,2.30,7.190000e+08,5.339000e+09
2021,3.94,1.210000e+09,6.319000e+09
2022,4.08,1.254000e+09,6.848000e+09


In [19]:
pd.options.display.float_format = '{:.2%}'.format
eps_df['epsdiluted'].pct_change(periods=1).fillna(0)


2018     0.00%
2019   247.42%
2020   -31.75%
2021    71.30%
2022     3.55%
Name: epsdiluted, dtype: float64

In [22]:
type(eps_df.columns[0])

str

In [20]:
import os
import pathlib

In [22]:
arr = os.listdir('D:\lianz\Desktop\Python\personal_projects\personal_finance\\balance-sheet-statement')

In [23]:
list_1 = [
  "UAL",
  "AF.PA",
  "T",
  "JNJ",
  "DIS",
  "AAPL",
  "TSLA",
  "JPM",
  "GOOG",
]

list_2 = [
  "AMD",
  "INGA.NV",
  "SHEL.L",
  "SMSN.L",
  "WMT",
  "NVDA",
  "WBD",
  "MMM",
  "AMZN",
  "COST"
]

In [24]:
[i for i in list_1 if i not in list_2]

['UAL', 'AF.PA', 'T', 'JNJ', 'DIS', 'AAPL', 'TSLA', 'JPM', 'GOOG']

In [27]:
def generate_key_metrics(financial_statement, list_of_metrics):
    l = []
    dict_holder = {}
    columns = financial_statement[0].keys()

    for n in list_of_metrics:

        if n in columns:
            for i, x in enumerate(financial_statement[::-1]):
                dict_holder[x['calendarYear']] = x[f'{n}']

            l.append(dict_holder)
            dict_holder = {}

        else:
            pass

    df = pd.DataFrame.from_records(
        l, index=[items for items in list_of_metrics if items in columns])

    # df.style.format({f"{df}": "{:,}"})

    return df

In [25]:

def read_statement(filepath, mode: list(['r','w','r+','w+'])):
    with open(filepath, f'{mode}') as f:
        statement = json.load(f)

        return statement

In [26]:
terms_interested = {'Revenue': 'revenue',
                    'Gross margin%': 'grossProfitRatio',
                    'Operating Income': 'operatingIncome',
                    'Operating Margin %': 'operatingIncomeRatio',
                    'Net Income': 'netIncome',
                    'Net Income Margin': 'netIncomeRatio',
                    'Earnings per Share': 'epsdiluted',
                    'Shares Oustanding (diluted)': 'weightedAverageShsOutDil',
                    'Dividends': 'dividendsPaid',
                    'Operating Cash Flow': 'operatingCashFlow',
                    'Cap Spending': 'capitalExpenditure',
                    'Free Cash Flow': 'freeCashFlow',
                    'Free Cash Flow per Share': 'freeCashFlow',
                    'Working Capital': 'totalCurrentAssets - totalCurrentLiabilities',
                    'Net Debt': 'netDebt'
                    }

In [27]:
pd.concat([generate_key_metrics(read_statement(f'D:\lianz\Desktop\Python\data_science_discovery\personal_finance\{x}\{"AAPL"}.json','r'), terms_interested.values()) for x in company_statements],axis=0)

NameError: name 'generate_key_metrics' is not defined

#### Append if year not in json file

In [28]:
file = pathlib.Path(
                f'D:\lianz\Desktop\Python\personal_projects\personal_finance\{company_statements[0]}/TSLA.json')
file

WindowsPath('D:/lianz/Desktop/Python/personal_projects/personal_finance/income-statement/TSLA.json')

In [29]:
def selectquote(ticker, statement):
    r = requests.get(
        f"https://financialmodelingprep.com/api/v3/{statement}/{ticker}?limit=120&apikey=eb29218df82acef0486b5c014ccec868")
    r = r.json()
    return r

In [57]:
new_download = selectquote('WDC', company_statements[0])

In [58]:
new_download[-1]

{'date': '1986-06-30',
 'symbol': 'WDC',
 'reportedCurrency': 'USD',
 'cik': '0000106040',
 'fillingDate': '1986-06-30',
 'acceptedDate': '1986-06-29 20:00:00',
 'calendarYear': '1986',
 'period': 'FY',
 'revenue': 277600000,
 'costOfRevenue': 164000000,
 'grossProfit': 113600000,
 'grossProfitRatio': 0.4092219020172911,
 'researchAndDevelopmentExpenses': 0,
 'generalAndAdministrativeExpenses': 0,
 'sellingAndMarketingExpenses': 0,
 'sellingGeneralAndAdministrativeExpenses': 70100000,
 'otherExpenses': 12900000,
 'operatingExpenses': 83000000,
 'costAndExpenses': 247000000,
 'interestIncome': 0,
 'interestExpense': 0,
 'depreciationAndAmortization': 12900000,
 'ebitda': 40700000,
 'ebitdaratio': 0.14661383285302593,
 'operatingIncome': 30600000,
 'operatingIncomeRatio': 0.11023054755043228,
 'totalOtherIncomeExpensesNet': -2800000,
 'incomeBeforeTax': 27800000,
 'incomeBeforeTaxRatio': 0.10014409221902017,
 'incomeTaxExpense': 7200000,
 'netIncome': 20600000,
 'netIncomeRatio': 0.07420

In [108]:
with open(f'D:\\lianz\\Desktop\\Python\\personal_projects\\personal_finance\\{company_statements[0]}\\WDC.json', 'r+') as f:
    loaded = json.load(f)

not_in = [i for i in new_download if i not in loaded]
len(not_in)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [112]:
with open(f'D:\\lianz\\Desktop\\Python\\personal_projects\\personal_finance\\{company_statements[0]}\\WDC.json', 'w+') as f:
    if len(not_in) == 0:
        pass
    else:
        new_adjusted = loaded.append([i for i in new_download if i not in loaded])
        json.dump(new_adjusted, f, indent=2)
    json.dump(new_download, f, indent=2)